In [1]:
import pandas as pd
import numpy as np

In [2]:
def contains_total(data_list):
    # Iterate over each element in the list
    for item in data_list:
        # Check if "TOTAL" is in the current item (case-insensitive)
        if "TOTAL" in item.upper():
            return True
    return False

def is_multiple_of_15(time_str):
    hour, minute = map(int, time_str.split(':'))
    return minute % 15 == 0
    
def read_excel_until_total(file_path, start_row=0):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Initialize a list to store data from all sheets
    all_data = []

    # Iterate through each sheet in the Excel file
    for i, sheet_name in enumerate(xls.sheet_names):
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name,header=0)

        # # Filter rows starting from `start_row`
        filtered_data = []

        for index, row in df.iterrows():
            if contains_total(row.astype(str).values):
                break
            filtered_data.append(row)

        # Convert the list of rows back into a DataFrame
        if filtered_data:  # Proceed only if there is data to add
            sheet_data = pd.DataFrame(filtered_data)
            sheet_data['SHEET NAME'] = sheet_name  # Add the sheet name to the DataFrame

            # Append the filtered data from this sheet to all_data
            all_data.append(sheet_data)

    # Concatenate all the data from all sheets
    result_df = pd.concat(all_data, ignore_index=True)
    
    if 'Unnamed: 6' in result_df.columns:
        result_df = result_df.drop(columns=['Unnamed: 6'])

    # Mengubah nama kolom menjadi huruf besar
    result_df.columns = result_df.columns.str.upper()
 
    # Mengubah format waktu
    result_df['WAKTU'] = result_df['WAKTU'].str[:5]
    return result_df

def penyesuaian_format(df):
    # Memastikan bahwa SHEET NAME diformat dengan dua digit (misalnya '01', '02', ..., '31')
    df['SHEET NAME'] = df['SHEET NAME'].str.zfill(2)
    
    # Menambahkan tanggal, bulan, dan tahun ke waktu
    df['WAKTU'] = thn_bln + df['SHEET NAME'] + ' ' + df['WAKTU']  # Tambahkan tanggal default
    
    # Mengubah kolom 'WAKTU' menjadi datetime dengan format lengkap
    df['WAKTU'] = pd.to_datetime(df['WAKTU'], format='%Y-%m-%d %H:%M')
    
    # Filter data berdasarkan apakah menit adalah kelipatan dari 15
    df = df[df['WAKTU'].dt.minute % 15 == 0]
    return df

def mencari_data_hilang(df, nama_file):    
    # Menghasilkan rentang tanggal untuk bulan yang ditentukan
    month_start = pd.Timestamp(f'{thn_bln}01')
    month_end = month_start + pd.offsets.MonthEnd(0)
    
    # Membuat rentang tanggal untuk seluruh bulan
    dates_range = pd.date_range(start=month_start, end=month_end, freq='D')
    
    # Inisialisasi list untuk menyimpan DataFrame dari waktu yang hilang
    missing_times_list = []
    
    # Loop melalui setiap hari dalam rentang bulan
    for date in dates_range:
        day = date.date()
        
        # Filter data untuk hari ini
        daily_data = df[df['WAKTU'].dt.date == day]
        
        # Buat rentang waktu yang lengkap untuk hari ini
        full_daily_range = pd.date_range(start=f'{day} 00:00', end=f'{day} 23:45', freq='15T')
        
        # Identifikasi waktu yang hilang
        missing_daily_times = full_daily_range[~full_daily_range.isin(daily_data['WAKTU'])]
        
        # Jika ada waktu yang hilang, tambahkan ke list missing_times_list
        if not missing_daily_times.empty:
            missing_times_df = pd.DataFrame({
                'WAKTU': missing_daily_times,
                'MOTOR': np.nan,
                'MOBIL': np.nan,
                'TRUK/BUS': np.nan,
                'JUMLAH': np.nan,
                'SHEET NAME': str(day.day).zfill(2),
                'INTERVAL': np.nan,
                'UPPKB': np.nan
                
            })
            missing_times_list.append(missing_times_df)
    
    # Menggabungkan semua DataFrame dari missing_times_list menjadi satu DataFrame
    missing_times = pd.concat(missing_times_list, ignore_index=True) if missing_times_list else pd.DataFrame()
    
    # Menampilkan data yang hilang
    if missing_times.empty:
        print(f"Bulan {nama_file[:2]} Tidak ada data yang hilang.")
    else:
        print(f"Bulan {nama_file[:2]} Ada data yang hilang")
        # print(missing_times)
    
    return missing_times, df

def pisahkan_kolom_waktu(df):
    # Pastikan kolom WAKTU sudah dalam format datetime
    df['WAKTU'] = pd.to_datetime(df['WAKTU'])

    # Pisahkan kolom WAKTU menjadi beberapa kolom
    df['TAHUN'] = df['WAKTU'].dt.year
    df['BULAN'] = df['WAKTU'].dt.month
    df['TANGGAL'] = df['WAKTU'].dt.day
    df['HARI'] = df['WAKTU'].dt.day_name()  # Ini akan memberikan nama hari dalam format string
    df['JAM'] = df['WAKTU'].dt.strftime('%H:%M')

    # Daftar kolom yang diinginkan dalam urutan baru
    new_column_order = [
        'WAKTU', 'TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM', 
        'MOTOR', 'MOBIL', 'TRUK/BUS', 'TOTAL',
        'SHEET NAME','INTERVAL', 'UPPKB'
    ]
    
    # Menyusun ulang kolom berdasarkan urutan yang diinginkan
    df = df.reindex(columns=new_column_order)
    return df
    
def hapus_kolom(df):
    # Misalkan df adalah DataFrame yang sudah ada
    df = df.drop(columns=['SHEET NAME','INTERVAL', 'UPPKB'])
    return df

def perhitungan_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.2
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])

    return df

In [3]:
# data files
nama_files = ['01 Januari 2024', '02 Februari 2024', '03 Maret 2024', '04 April 2024']


# Inisialisasi list untuk menampung DataFrame
all_dfs = []
all_miss = []

for nama_file in nama_files:
    bulan = nama_file.split()[0]
    tahun = nama_file.split()[2]
    thn_bln = f'{tahun}-{bulan}-'
    
    nama_file = nama_file + '.xlsx'
    file_path = '../Data/DataLHR/Data Losarang/2024/' + nama_file 
    
    # Read the Excel file from the specified row until 'Total' is encountered
    result = read_excel_until_total(file_path, start_row=11)
    result = penyesuaian_format(result)
    missing_times, result = mencari_data_hilang(result, nama_file)
    missing_times = pisahkan_kolom_waktu(missing_times)
    missing_times = hapus_kolom(missing_times)
    missing_times = perhitungan_smp(missing_times)
    result = pisahkan_kolom_waktu(result)
    result = hapus_kolom(result)
    result = perhitungan_smp(result)
    
    df = result.copy()

    # Tambahkan DataFrame ke list
    all_dfs.append(result)
    all_miss.append(missing_times)
    
    # save the result to a new Excel file
    df.to_excel('../Data/Preprocessing/Data Losarang/2024/1_input_'+nama_file, index=False)
    missing_times.to_excel('../Data/Preprocessing/Data Losarang/2024/1_missing_times_'+nama_file, index=False)
    

# gabungkan data
df_combined = pd.concat(all_dfs)
df_combined_miss = pd.concat(all_miss)

Bulan 01 Ada data yang hilang
Bulan 02 Ada data yang hilang
Bulan 03 Ada data yang hilang
Bulan 04 Ada data yang hilang


In [4]:
df_combined = df_combined.rename(columns={'TOTAL': 'JUMLAH'})
df_combined.head(20)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,120.0,15.6,30,30.0,76.0
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,138.0,19.4,32,22.5,74.0
2,2024-01-01 00:30:00,2024,1,1,Monday,00:30,98,41,7,146.0,19.6,41,17.5,78.0
3,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,131.0,17.6,31,30.0,79.0
4,2024-01-01 01:00:00,2024,1,1,Monday,01:00,56,28,12,96.0,11.2,28,30.0,69.0
5,2024-01-01 01:15:00,2024,1,1,Monday,01:15,61,34,17,112.0,12.2,34,42.5,89.0
6,2024-01-01 01:30:00,2024,1,1,Monday,01:30,60,38,30,128.0,12.0,38,75.0,125.0
7,2024-01-01 01:45:00,2024,1,1,Monday,01:45,57,45,15,117.0,11.4,45,37.5,94.0
8,2024-01-01 02:00:00,2024,1,1,Monday,02:00,58,26,11,95.0,11.6,26,27.5,65.0
9,2024-01-01 02:15:00,2024,1,1,Monday,02:15,62,44,20,126.0,12.4,44,50.0,106.0


In [5]:
# Rename kolom
df_combined_miss = df_combined_miss.rename(columns={'TOTAL': 'JUMLAH'})
df_combined_miss.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-02 01:15:00,2024,1,2,Tuesday,01:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-02 02:00:00,2024,1,2,Tuesday,02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-02 05:30:00,2024,1,2,Tuesday,05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-02 06:30:00,2024,1,2,Tuesday,06:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-02 07:30:00,2024,1,2,Tuesday,07:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Simpan hasil gabungan ke file Excel
df_combined.to_excel('../Data/Preprocessing/Data Losarang/2024/1_input_losarang_2024.xlsx', index=False)
df_combined_miss.to_excel('../Data/Preprocessing/Data Losarang/2024/1_missing_times_losarang_2024.xlsx', index=False)